# Seurat object  
## Load required packages


In [ ]:
library(Seurat)
library(tidyverse)
library(ggplot2)
library("scales")



## Load seurat objects (RAW)


In [ ]:
line <- params$line



In [ ]:
diff_obj <- readRDS(file = paste0('./results/rds/samples/Filtered_', line, '_diff.rds'))
diff_obj


In [ ]:
prolif_obj <- readRDS(file = paste0('./results/rds/samples/Filtered_', line, '_prolif.rds'))
prolif_obj



##  <span style="color:#296d98;"> Parameters </span>  


In [ ]:
pc = 60



# Cell cycle scoring  
## Condition : differenciation 


In [ ]:
diff_obj.cellcycle <- NormalizeData(diff_obj)
diff_obj <- CellCycleScoring(diff_obj.cellcycle, s.features = cc.genes.updated.2019$s.genes, g2m.features = cc.genes.updated.2019$g2m.genes, set.ident = TRUE)


In [ ]:
as_tibble(diff_obj[[]]) %>%
  ggplot(aes(Phase, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3'))+ geom_bar()


as_tibble(diff_obj[[]]) %>%
  ggplot(aes(x=S.Score, y=G2M.Score, color=Phase)) + 
  scale_color_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3')) +
  geom_point() +
  coord_cartesian(xlim=c(-0.15,0.15), ylim=c(-0.15,0.15))



## Condition : proliferation  


In [ ]:
prolif_obj.cellcycle <- NormalizeData(prolif_obj)
prolif_obj <- CellCycleScoring(prolif_obj.cellcycle, s.features = cc.genes.updated.2019$s.genes, g2m.features = cc.genes.updated.2019$g2m.genes, set.ident = TRUE)


In [ ]:
as_tibble(prolif_obj[[]]) %>%
  ggplot(aes(Phase, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3'))+ geom_bar()


as_tibble(prolif_obj[[]]) %>%
  ggplot(aes(x=S.Score, y=G2M.Score, color=Phase)) + 
  scale_color_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3')) +
  geom_point() +
  coord_cartesian(xlim=c(-0.15,0.15), ylim=c(-0.15,0.15))


# Normalization    
## Condition : differentiation
### Zero counts  
See features without any counts left after the filtering.


In [ ]:
counts <- GetAssayData(diff_obj, assay = "RNA")
zero <- which(rowSums(counts) == 0 )
length(zero)

keep.genes <- which(rowSums(counts) != 0 )
diff_obj <- subset(diff_obj, features = keep.genes)
diff_obj


### SCTransform  
**SCTransform** : replace  `NormalizeData()`, `ScaleData()`, and `FindVariableFeatures()`.This procedure omits the need for heuristic steps including *pseudocount addition* or *log-transformation* and improves common downstream analytical tasks such as variable gene selection, dimensional reduction, and differential expression.  
During normalization, we can also remove confounding sources of variation, for example, mitochondrial mapping percentage. (not done here)

Corrected counts are obtained by setting the sequencing depth for all the cells to a fixed value and reversing the learned regularized negative-binomial regression model. 


In [ ]:
diff_obj <- SCTransform(diff_obj, vst.flavor = "v2", verbose = FALSE)



In [ ]:
tibble(
  pc95 = apply(diff_obj[["SCT"]]@data,2,quantile,0.95),
  measured = apply(diff_obj[["SCT"]]@data,2,function(x)(100*sum(x!=0))/length(x))
) -> normalisation.qc

normalisation.qc %>% 
  ggplot(aes(x=measured,y=pc95))+
  geom_point()+
  ggtitle("Normalisation of data")



### Save norm object  


In [ ]:
saveRDS(diff_obj, file = paste0("./results/rds/samples/Norm_", line,"_diff.rds"))



## Condition : proliferation
### Zero counts  
See features without any counts left after the filtering.


In [ ]:
counts <- GetAssayData(prolif_obj, assay = "RNA")
zero <- which(rowSums(counts) == 0 )
length(zero)

keep.genes <- which(rowSums(counts) != 0 )
prolif_obj <- subset(prolif_obj, features = keep.genes)
prolif_obj



### SCTransform  


In [ ]:
prolif_obj <- SCTransform(prolif_obj, vst.flavor = "v2", verbose = FALSE)



In [ ]:
tibble(
  pc95 = apply(prolif_obj[["SCT"]]@data,2,quantile,0.95),
  measured = apply(prolif_obj[["SCT"]]@data,2,function(x)(100*sum(x!=0))/length(x))
) -> normalisation.qc

normalisation.qc %>% 
  ggplot(aes(x=measured,y=pc95))+
  geom_point()+
  ggtitle("Normalisation of data")



### Save norm object  


In [ ]:
saveRDS(prolif_obj, file = paste0("./results/rds/samples/Norm_", line,"_prolif.rds"))




# QC metrics (differentiation)


In [ ]:
metadata <- diff_obj@meta.data




## Nb of reads  


In [ ]:
VlnPlot(diff_obj,features="nCount_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(diff_obj, features = "nCount_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nCount_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()



## Nb of genes  


In [ ]:
VlnPlot(diff_obj,features="nFeature_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(diff_obj, features = "nFeature_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nFeature_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()
    


In [ ]:
ggplot(diff_obj[[]]) + 
   geom_histogram(aes(nFeature_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()


## Gene expression  
We can pick the first 100 cells and look at the distributions of their expression values.


In [ ]:
apply(diff_obj@assays$SCT@data,1,mean) -> gene.expression
sort(gene.expression, decreasing = TRUE) -> gene.expression

as.tibble(
  diff_obj@assays$SCT@data[,1:100]
) %>%
  pivot_longer(
    cols=everything(),
    names_to="cell",
    values_to="expression"
  ) %>%
  ggplot(aes(x=expression, group=cell)) +
  geom_density() +
  coord_cartesian(ylim=c(0,0.6), xlim=c(0,3))



## MT genes  


In [ ]:
diff_obj[["percent.mt"]] <- PercentageFeatureSet(diff_obj, pattern = "^MT-")
VlnPlot(diff_obj, features = "percent.mt", group.by = "orig.ident", pt.size = 0)
RidgePlot(diff_obj, features = "percent.mt", group.by = "orig.ident")


In [ ]:
ggplot(diff_obj[[]]) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()



## Ribosomal genes  


In [ ]:
PercentageFeatureSet(diff_obj,pattern="^RP[LS]") -> diff_obj[["percent.ribosomal"]] 
VlnPlot(diff_obj, features = "percent.ribosomal", group.by = "orig.ident", pt.size = 0)
RidgePlot(diff_obj, features = "percent.ribosomal", group.by = "orig.ident")


In [ ]:
ggplot(diff_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()



## Largest gene  


In [ ]:
apply(
  diff_obj@assays$SCT@counts,
  2,
  max
) -> largest_count

apply(
  diff_obj@assays$SCT@counts,
  2,
  which.max
) -> largest_index

rownames(diff_obj)[largest_index] -> diff_obj$largest_gene

100 * largest_count / diff_obj$nCount_RNA -> diff_obj$percent.largest_gene

VlnPlot(diff_obj,features="percent.largest_gene", group.by = "orig.ident", pt.size = 0)
RidgePlot(diff_obj, features = "percent.largest_gene", group.by = "orig.ident")


In [ ]:
ggplot(diff_obj[[]]) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()



## FeatureScatter  


In [ ]:
FeatureScatter(diff_obj, feature1 = "nCount_RNA", feature2 = "percent.mt", group.by ="orig.ident")
FeatureScatter(diff_obj,feature1 = "nCount_RNA", feature2 = "percent.largest_gene",group.by ="orig.ident")

FeatureScatter(diff_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by ="orig.ident") + geom_smooth(method = 'lm')



## Cell cycle scoring  


In [ ]:
diff_obj <- CellCycleScoring(diff_obj, s.features = cc.genes.updated.2019$s.genes, g2m.features = cc.genes.updated.2019$g2m.genes, set.ident = TRUE)

as_tibble(diff_obj[[]]) %>%
  ggplot(aes(Phase, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3'))+ geom_bar()


as_tibble(diff_obj[[]]) %>%
  ggplot(aes(x=S.Score, y=G2M.Score, color=Phase)) + 
  scale_color_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3')) +
  geom_point() +
  coord_cartesian(xlim=c(-0.15,0.15), ylim=c(-0.15,0.15))



# QC metrics (proliferation)


In [ ]:
metadata <- prolif_obj@meta.data




## Nb of reads  


In [ ]:
VlnPlot(prolif_obj,features="nCount_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(prolif_obj, features = "nCount_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nCount_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()



## Nb of genes  


In [ ]:
VlnPlot(prolif_obj,features="nFeature_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(prolif_obj, features = "nFeature_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nFeature_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()
    


In [ ]:
ggplot(prolif_obj[[]]) + 
   geom_histogram(aes(nFeature_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()


## Gene expression  
We can pick the first 100 cells and look at the distributions of their expression values.


In [ ]:
apply(prolif_obj@assays$SCT@data,1,mean) -> gene.expression
sort(gene.expression, decreasing = TRUE) -> gene.expression

as.tibble(
  prolif_obj@assays$SCT@data[,1:100]
) %>%
  pivot_longer(
    cols=everything(),
    names_to="cell",
    values_to="expression"
  ) %>%
  ggplot(aes(x=expression, group=cell)) +
  geom_density() +
  coord_cartesian(ylim=c(0,0.6), xlim=c(0,3))



## MT genes  


In [ ]:
prolif_obj[["percent.mt"]] <- PercentageFeatureSet(prolif_obj, pattern = "^MT-")
VlnPlot(prolif_obj, features = "percent.mt", group.by = "orig.ident", pt.size = 0)
RidgePlot(prolif_obj, features = "percent.mt", group.by = "orig.ident")


In [ ]:
ggplot(prolif_obj[[]]) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()



## Ribosomal genes  


In [ ]:
PercentageFeatureSet(prolif_obj,pattern="^RP[LS]") -> prolif_obj[["percent.ribosomal"]] 
VlnPlot(prolif_obj, features = "percent.ribosomal", group.by = "orig.ident", pt.size = 0)
RidgePlot(prolif_obj, features = "percent.ribosomal", group.by = "orig.ident")


In [ ]:
ggplot(prolif_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()



## Largest gene  


In [ ]:
apply(
  prolif_obj@assays$SCT@counts,
  2,
  max
) -> largest_count

apply(
  prolif_obj@assays$SCT@counts,
  2,
  which.max
) -> largest_index

rownames(prolif_obj)[largest_index] -> prolif_obj$largest_gene

100 * largest_count / prolif_obj$nCount_RNA -> prolif_obj$percent.largest_gene

VlnPlot(prolif_obj,features="percent.largest_gene", group.by = "orig.ident", pt.size = 0)
RidgePlot(prolif_obj, features = "percent.largest_gene", group.by = "orig.ident")


In [ ]:
ggplot(prolif_obj[[]]) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()



## FeatureScatter  


In [ ]:
FeatureScatter(prolif_obj, feature1 = "nCount_RNA", feature2 = "percent.mt", group.by ="orig.ident")
FeatureScatter(prolif_obj,feature1 = "nCount_RNA", feature2 = "percent.largest_gene",group.by ="orig.ident")

FeatureScatter(prolif_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by ="orig.ident") + geom_smooth(method = 'lm')



## Cell cycle scoring  


In [ ]:
prolif_obj <- CellCycleScoring(prolif_obj, s.features = cc.genes.updated.2019$s.genes, g2m.features = cc.genes.updated.2019$g2m.genes, set.ident = TRUE)

as_tibble(prolif_obj[[]]) %>%
  ggplot(aes(Phase, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3'))+ geom_bar()


as_tibble(prolif_obj[[]]) %>%
  ggplot(aes(x=S.Score, y=G2M.Score, color=Phase)) + 
  scale_color_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3')) +
  geom_point() +
  coord_cartesian(xlim=c(-0.15,0.15), ylim=c(-0.15,0.15))


# Identify high variable features  
*Included in the `SCTransform` function.*

## Condition : differenciation   


In [ ]:
# identify the 10 most highly variable genes
top10 <- head(VariableFeatures(diff_obj), 10)

# plot the variable features 
plot <- VariableFeaturePlot(diff_obj)
LabelPoints(plot = plot, points = head(top10,10), repel = TRUE)


In [ ]:
as_tibble(HVFInfo(diff_obj),rownames = "Gene") -> variance.data

variance.data %>% 
  mutate(hypervariable=Gene %in% VariableFeatures(diff_obj)
) -> variance.data

variance.data %>% 
  ggplot(aes(log(mean),log(variance),color=hypervariable)) + 
  geom_point() + 
  scale_color_manual(values=c("black","red"))



## Condition : proliferation  


In [ ]:
# identify the 10 most highly variable genes
top10 <- head(VariableFeatures(prolif_obj), 10)

# plot the variable features 
plot <- VariableFeaturePlot(prolif_obj)
LabelPoints(plot = plot, points = head(top10,10), repel = TRUE)


We can plot out a graph of the variance vs mean and highlight the selected genes so we can see whether we think we’re likely to capture what we need.    



In [ ]:
as_tibble(HVFInfo(prolif_obj),rownames = "Gene") -> variance.data

variance.data %>% 
  mutate(hypervariable=Gene %in% VariableFeatures(prolif_obj)
) -> variance.data

variance.data %>% 
  ggplot(aes(log(mean),log(variance),color=hypervariable)) + 
  geom_point() + 
  scale_color_manual(values=c("black","red"))


# Scaling the data  
*Included in the `SCTransform` function.*

**ScaleData** :

* Shifts the expression of each gene, so that the mean expression across cells is 0
* Scales the expression of each gene, so that the variance across cells is 1 
    + This step gives equal weight in downstream analyses, so that highly-expressed genes do not 

# Integrate data
Pearson residuals are used for identifying variable features, PCA, dimensionality reduction and integration.
The corrected counts should be used for DE/visualization of marker genes.


In [ ]:
seurat.list <-list(diff_obj, prolif_obj)

# select features that are repeatedly variable across datasets for integration
features <- SelectIntegrationFeatures(object.list = seurat.list, nfeatures = 3000)
seurat.list <- PrepSCTIntegration(object.list = seurat.list, anchor.features = features)

anchors <- FindIntegrationAnchors(object.list = seurat.list, normalization.method = "SCT", anchor.features = features)


In [ ]:
seurat_obj <- IntegrateData(anchorset = anchors, normalization.method = "SCT")
DefaultAssay(seurat_obj) <- "integrated"

seurat_obj
table(seurat_obj$orig.ident)


In [ ]:
counts <- GetAssayData(seurat_obj, assay = "integrated")
print(length(rownames(counts)))

counts <- GetAssayData(seurat_obj, assay = "RNA")
print(length(rownames(counts)))

counts <- GetAssayData(seurat_obj, assay = "SCT")
print(length(rownames(counts)))


## Zero counts  
See features without any counts left after the filtering.


In [ ]:
counts <- GetAssayData(seurat_obj, assay = "SCT")
zero <- which(rowSums(counts) == 0 )
length(zero)



# QC metrics (after integration) 


In [ ]:
metadata <- seurat_obj@meta.data




## Nb of reads  


In [ ]:
VlnPlot(seurat_obj,features="nCount_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "nCount_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nCount_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()



## Nb of genes  


In [ ]:
VlnPlot(seurat_obj,features="nFeature_RNA", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "nFeature_RNA", group.by = "orig.ident")

metadata %>% 
  ggplot(aes( x=nFeature_RNA,y=orig.ident, fill= orig.ident)) + 
	geom_boxplot() + 
	theme_classic()+
  scale_x_log10()
    


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(nFeature_RNA), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 50) + 
  ggtitle("Distribution of nFeature_RNA ") + NoLegend()


## Gene expression  
We can pick the first 100 cells and look at the distributions of their expression values.


In [ ]:
apply(seurat_obj@assays$SCT@data,1,mean) -> gene.expression
sort(gene.expression, decreasing = TRUE) -> gene.expression

as.tibble(
  seurat_obj@assays$SCT@data[,1:100]
) %>%
  pivot_longer(
    cols=everything(),
    names_to="cell",
    values_to="expression"
  ) %>%
  ggplot(aes(x=expression, group=cell)) +
  geom_density() +
  coord_cartesian(ylim=c(0,0.6), xlim=c(0,3))


## MT genes  
*calculating pct on RNA assay (raw)*


In [ ]:
seurat_obj[["percent.mt"]] <- PercentageFeatureSet(seurat_obj, pattern = "^MT-", assay = 'RNA')
VlnPlot(seurat_obj, features = "percent.mt", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.mt", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.mt), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.5) + 
  ggtitle("Distribution of Percentage Mitochondrion") + NoLegend()



## Ribosomal genes  


In [ ]:
PercentageFeatureSet(seurat_obj,pattern="^RP[LS]", assay = 'RNA') -> seurat_obj[["percent.ribosomal"]] 
VlnPlot(seurat_obj, features = "percent.ribosomal", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.ribosomal", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.ribosomal), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Ribosomal") + NoLegend()



## Largest gene  


In [ ]:
apply(
  seurat_obj@assays$SCT@counts,
  2,
  max
) -> largest_count

apply(
  seurat_obj@assays$SCT@counts,
  2,
  which.max
) -> largest_index

rownames(seurat_obj)[largest_index] -> seurat_obj$largest_gene

100 * largest_count / seurat_obj$nCount_RNA -> seurat_obj$percent.largest_gene

VlnPlot(seurat_obj,features="percent.largest_gene", group.by = "orig.ident", pt.size = 0)
RidgePlot(seurat_obj, features = "percent.largest_gene", group.by = "orig.ident")


In [ ]:
ggplot(seurat_obj[[]]) + 
   geom_histogram(aes(percent.largest_gene), 
                  color = "#558bdc", fill= "#173664",
                  binwidth = 0.7) + 
  ggtitle("Distribution of Percentage Largest Gene") + NoLegend()



## FeatureScatter  


In [ ]:
FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "percent.mt", group.by ="orig.ident")
FeatureScatter(seurat_obj,feature1 = "nCount_RNA", feature2 = "percent.largest_gene",group.by ="orig.ident")

FeatureScatter(seurat_obj, feature1 = "nCount_RNA", feature2 = "nFeature_RNA",group.by ="orig.ident") + geom_smooth(method = 'lm')



## Cell cycle scoring  


In [ ]:
seurat_obj <- CellCycleScoring(seurat_obj, s.features = cc.genes.updated.2019$s.genes, g2m.features = cc.genes.updated.2019$g2m.genes, set.ident = TRUE)

seurat_obj@meta.data %>%
  group_by(orig.ident,Phase) %>%
  count() %>%
  group_by(orig.ident) %>%
  mutate(percent=100*n/sum(n)) %>%
  ungroup() %>%
  ggplot(aes(x=orig.ident,y=percent, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3')) +
  geom_col() +
  ggtitle("Percentage of cell cycle phases per sample")

as_tibble(seurat_obj[[]]) %>%
  ggplot(aes(Phase, fill=Phase)) +
  scale_fill_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3'))+ geom_bar()


as_tibble(seurat_obj[[]]) %>%
  ggplot(aes(x=S.Score, y=G2M.Score, color=Phase)) + 
  scale_color_manual( values=c(G1='azure4',G2M='dodgerblue',S='tomato3')) +
  geom_point() +
  coord_cartesian(xlim=c(-0.15,0.15), ylim=c(-0.15,0.15))


# Identify high variable features  
*Included in the `SCTransform` function.*


In [ ]:
# identify the 10 most highly variable genes
top10 <- head(VariableFeatures(seurat_obj), 10)

# plot the variable features 
plot <- VariableFeaturePlot(seurat_obj)
LabelPoints(plot = plot, points = head(top10,10), repel = TRUE)


We can plot out a graph of the variance vs mean and highlight the selected genes so we can see whether we think we’re likely to capture what we need.    



In [ ]:
as_tibble(HVFInfo(seurat_obj),rownames = "Gene") -> variance.data

variance.data %>% 
  mutate(hypervariable=Gene %in% VariableFeatures(seurat_obj)
) -> variance.data

variance.data %>% 
  ggplot(aes(log(mean),log(variance),color=hypervariable)) + 
  geom_point() + 
  scale_color_manual(values=c("black","red"))


# Linear dimensional reduction  
## PCA  


In [ ]:
seurat_obj <- RunPCA(seurat_obj, verbose = FALSE)
VizDimLoadings(seurat_obj, dims = 1:2, reduction = "pca")


In [ ]:
mat <- Seurat::GetAssayData(seurat_obj, assay = "SCT", slot = "scale.data") ## instead of RNA
pca <- seurat_obj[["pca"]]

# Get the total variance:
total_variance <- sum(matrixStats::rowVars(mat))
eigValues = (pca@stdev)^2  ## EigenValues

varExplained = eigValues / total_variance

pc1 = percent(varExplained[1], accuracy = 0.01)
pc2 = percent(varExplained[2], accuracy = 0.01)


In [ ]:
DimPlot(seurat_obj, reduction = "pca", group.by = "orig.ident") + xlab(paste0("PC_1 : ", pc1)) + ylab(paste0("PC_2 :", pc2 ))
DimPlot(seurat_obj, reduction = "pca", group.by = "Phase", cols = c(G1='azure4',G2M='dodgerblue',S='tomato3'))

DimPlot(seurat_obj, reduction = "pca", group.by = "orig.ident", label.size = 4, label = TRUE)



### Split by condition


In [ ]:
DimPlot(seurat_obj, reduction = "pca", group.by="orig.ident", split.by = "condition")
DimPlot(seurat_obj, reduction = "pca", group.by = "orig.ident", split.by = "condition", label.size = 4, label = TRUE)+ NoLegend()
DimPlot(seurat_obj, reduction = "pca", group.by="orig.ident", split.by = "condition")+ NoLegend()



## Save seurat object  


In [ ]:
saveRDS(seurat_obj, file = paste0("./results/rds/samples/Norm_", line,".rds"))



## Determine the dimensionality of the dataset  
how many components should we choose to include?  

**JackStrawPlot**  
*significant* PCs as those who have a strong enrichment of **low p-value**

* dashed line : uniform distribution
* solid curve above dash line : strong enrichments of features with low p-values  

*Cannot run Jackstraw plot on SCT data*  


In [ ]:
seurat_obj <- JackStraw(seurat_obj, num.replicate = 100, dims = pc)
jack.data <- seurat_obj 

seurat_obj <- ScoreJackStraw(seurat_obj, dims = 1:pc)

JackStrawPlot(seurat_obj, dims = 1:pc)


**ElbowPlot**  
ranking of principle components based on the percentage of variance explained by each one 


In [ ]:
ElbowPlot(seurat_obj, ndims = pc)

